In [1]:
class CuboRubik:
    def __init__(self, estado=None):
        if estado is None:
            self.estado = {
                'U': [['W', 'W', 'W'],
                      ['W', 'W', 'W'],
                      ['W', 'W', 'W']],
                'D': [['Y', 'Y', 'Y'],
                      ['Y', 'Y', 'Y'],
                      ['Y', 'Y', 'Y']],
                'F': [['G', 'G', 'G'],
                      ['G', 'G', 'G'],
                      ['G', 'G', 'G']],
                'B': [['B', 'B', 'B'],
                      ['B', 'B', 'B'],
                      ['B', 'B', 'B']],
                'L': [['O', 'O', 'O'],
                      ['O', 'O', 'O'],
                      ['O', 'O', 'O']],
                'R': [['R', 'R', 'R'],
                      ['R', 'R', 'R'],
                      ['R', 'R', 'R']]
            }
        else:
            self.estado = estado

    def mostrar_cubo(self):
        colores = {
            'W': '\033[1;37mW\033[0m',  # Blanco
            'Y': '\033[1;33mY\033[0m',  # Amarillo
            'G': '\033[1;32mG\033[0m',  # Verde
            'B': '\033[1;34mB\033[0m',  # Azul
            'O': '\033[1;22mO\033[0m',  # Naranja
            'R': '\033[1;31mR\033[0m'   # Rojo
        }

        for fila in self.estado['U']:
            print("      " + " ".join([colores[elem] for elem in fila]))
        for fila_L, fila_F, fila_R, fila_B in zip(self.estado['L'], self.estado['F'], self.estado['R'], self.estado['B']):
            print(" ".join([colores[fila_L[0]], colores[fila_L[1]], colores[fila_L[2]]]) +
                  "  " +
                  " ".join([colores[fila_F[0]], colores[fila_F[1]], colores[fila_F[2]]]) +
                  "  " +
                  " ".join([colores[fila_R[0]], colores[fila_R[1]], colores[fila_R[2]]]) +
                  "  " +
                  " ".join([colores[fila_B[0]], colores[fila_B[1]], colores[fila_B[2]]]))
        for fila in self.estado['D']:
            print("      " + " ".join([colores[elem] for elem in fila]))
        print()

    def girar_cara(self, cara, sentido='d'):
        if sentido == 'd':
            self.estado[cara] = [list(x) for x in zip(*self.estado[cara][::-1])]
        elif sentido == 'i':
            self.estado[cara] = [list(x) for x in zip(*self.estado[cara])][::-1]

    def girar_lado(self, lado, sentido='d'):
        if lado == 'U':
            self.girar_cara('U', sentido)
            temp = self.estado['F'][0][:]
            self.estado['F'][0] = self.estado['L'][0]
            self.estado['L'][0] = self.estado['B'][0][::-1]
            self.estado['B'][0] = self.estado['R'][0]
            self.estado['R'][0] = temp[::-1]
        elif lado == 'D':
            self.girar_cara('D', sentido)
            temp = self.estado['F'][2][:]
            self.estado['F'][2] = self.estado['R'][2]
            self.estado['R'][2] = self.estado['B'][2][::-1]
            self.estado['B'][2] = self.estado['L'][2]
            self.estado['L'][2] = temp[::-1]
        elif lado == 'F':
            self.girar_cara('F', sentido)
            temp = self.estado['U'][2][:]
            self.estado['U'][2] = self.estado['R'][0][::-1]
            self.estado['R'][0] = self.estado['D'][0]
            self.estado['D'][0] = self.estado['L'][2][::-1]
            self.estado['L'][2] = temp
        elif lado == 'B':
            self.girar_cara('B', sentido)
            temp = self.estado['U'][0][:]
            self.estado['U'][0] = self.estado['L'][0][::-1]
            self.estado['L'][0] = self.estado['D'][2]
            self.estado['D'][2] = self.estado['R'][2][::-1]
            self.estado['R'][2] = temp
        elif lado == 'L':
            self.girar_cara('L', sentido)
            temp = [fila[0] for fila in self.estado['U']]
            for i, fila in enumerate(self.estado['U']):
                fila[0] = self.estado['B'][i][0]
                self.estado['B'][i][0] = self.estado['D'][2 - i][2]
                self.estado['D'][2 - i][2] = self.estado['F'][2 - i][0]
                self.estado['F'][2 - i][0] = temp[i]
        elif lado == 'R':
            self.girar_cara('R', sentido)
            temp = [fila[2] for fila in self.estado['U']]
            for i, fila in enumerate(self.estado['U']):
                fila[2] = self.estado['F'][i][2]
                self.estado['F'][i][2] = self.estado['D'][i][0]
                self.estado['D'][i][0] = self.estado['B'][2 - i][2]
                self.estado['B'][2 - i][2] = temp[i]


In [2]:
if __name__ == "__main__":
    cubo = CuboRubik()
    cubo.mostrar_cubo()
    cubo.girar_lado('U', 'd')
    cubo.mostrar_cubo()
    cubo.girar_lado('L', 'i')
    cubo.mostrar_cubo()

      W W W
      W W W
      W W W
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
      Y Y Y
      Y Y Y
      Y Y Y

      W W W
      W W W
      W W W
B B B  O O O  G G G  R R R
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
      Y Y Y
      Y Y Y
      Y Y Y

      R W W
      B W W
      B W W
B O O  W O O  G G G  Y R R
B O O  W G G  R R R  Y B B
B O O  W G G  R R R  Y B B
      Y Y O
      Y Y G
      Y Y G



In [3]:
from queue import PriorityQueue
from copy import deepcopy

class NodoCubo:
    def __init__(self, estado, costo_g, padre=None):
        self.estado = estado
        self.costo_g = costo_g
        self.padre = padre

    def __lt__(self, other):
        return self.costo_g < other.costo_g

    def __eq__(self, other):
        return self.estado == other.estado

    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.estado.cubo.values()))

def heuristica_patrones(estado_actual):
    patrones_objetivo = {
        'U': [['W', 'W', 'W'],
              ['W', 'W', 'W'],
              ['W', 'W', 'W']],
        'D': [['Y', 'Y', 'Y'],
              ['Y', 'Y', 'Y'],
              ['Y', 'Y', 'Y']],
        'F': [['G', 'G', 'G'],
              ['G', 'G', 'G'],
              ['G', 'G', 'G']],
        'B': [['B', 'B', 'B'],
              ['B', 'B', 'B'],
              ['B', 'B', 'B']],
        'L': [['O', 'O', 'O'],
              ['O', 'O', 'O'],
              ['O', 'O', 'O']],
        'R': [['R', 'R', 'R'],
              ['R', 'R', 'R'],
              ['R', 'R', 'R']]
    }
    
    patrones_encontrados = 0
    
    for cara, patron_objetivo in patrones_objetivo.items():
        cara_actual = estado_actual.estado[cara]
        if cara_actual == patron_objetivo:
            patrones_encontrados += 1
    
    costo_heuristico = 6 - patrones_encontrados
    
    return costo_heuristico


def generar_sucesores(nodo):
    sucesores = []
    movimientos = ['U', 'D', 'F', 'B', 'L', 'R']

    for movimiento in movimientos:
        for sentido in ['d', 'i']:
            nuevo_estado = deepcopy(nodo.estado)
            nuevo_estado.girar_lado(movimiento, sentido)
            nuevo_nodo = NodoCubo(nuevo_estado, nodo.costo_g + 1, nodo)
            sucesores.append(nuevo_nodo)

    return sucesores

def resolver_cubo(cubo_inicial, cubo_objetivo):
    frontera = PriorityQueue()
    nodo_inicial = NodoCubo(cubo_inicial, 0)
    frontera.put(nodo_inicial)
    visitados = set()
    paso = 0

    cubo_objetivo_tupla = tuple(tuple(row) for row in cubo_objetivo.estado.values())

    while not frontera.empty():
        nodo_actual = frontera.get()

        costo_h = heuristica_patrones(nodo_actual.estado)
        print("Paso", paso, ":", "Costo G =", nodo_actual.costo_g, "Costo H =", costo_h)
        nodo_actual.estado.mostrar_cubo()

        paso += 1

        cubo_actual_tupla = tuple(tuple(row) for row in nodo_actual.estado.estado.values())

        if cubo_actual_tupla == cubo_objetivo_tupla:
            return reconstruir_camino(nodo_actual)

        visitados.add(nodo_actual.estado)

        for sucesor in generar_sucesores(nodo_actual):
            if sucesor.estado not in visitados:
                sucesor.costo_h = costo_h 
                sucesor.costo_f = sucesor.costo_g + sucesor.costo_h
                frontera.put(sucesor)

    return None

def reconstruir_camino(nodo):
    camino = []
    while nodo:
        camino.append(nodo.estado)
        nodo = nodo.padre
    camino.reverse()
    return camino

cubo_desordenado = CuboRubik(
    {
        'U': [['W', 'W', 'W'],
              ['W', 'W', 'W'],
              ['W', 'W', 'W']],
        'D': [['Y', 'Y', 'Y'],
              ['Y', 'Y', 'Y'],
              ['Y', 'Y', 'Y']],
        'F': [['O', 'O', 'O'],
              ['G', 'G', 'G'],
              ['G', 'G', 'G']],
        'B': [['R', 'R', 'R'],
              ['B', 'B', 'B'],
              ['B', 'B', 'B']],
        'L': [['B', 'B', 'B'],
              ['O', 'O', 'O'],
              ['O', 'O', 'O']],
        'R': [['G', 'G', 'G'],
              ['R', 'R', 'R'],
              ['R', 'R', 'R']]
    }
)

cubo_objetivo = CuboRubik()
camino_solucion = resolver_cubo(cubo_desordenado, cubo_objetivo)
if camino_solucion:
    print("Solución encontrada:", camino_solucion)
else:
    print("No se encontró solución.")



Paso 0 : Costo G = 0 Costo H = 4
      W W W
      W W W
      W W W
B B B  O O O  G G G  R R R
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
      Y Y Y
      Y Y Y
      Y Y Y

Paso 1 : Costo G = 1 Costo H = 4
      W W W
      W W W
      W W W
R R R  B B B  O O O  G G G
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
      Y Y Y
      Y Y Y
      Y Y Y

Paso 2 : Costo G = 1 Costo H = 4
      W W W
      W W W
      W W W
B B B  O O O  G G G  R R R
O O O  G G G  R R R  B B B
G G G  R R R  B B B  O O O
      Y Y Y
      Y Y Y
      Y Y Y

Paso 3 : Costo G = 1 Costo H = 6
      B B B
      W W W
      W W W
Y Y Y  O O O  G G G  B B R
O O O  G G G  R R R  B B R
O O O  G G G  W W W  B B R
      Y Y Y
      Y Y Y
      R R R

Paso 4 : Costo G = 1 Costo H = 6
      W W O
      W W G
      W W G
B B B  O O Y  G R R  R R W
O O O  G G Y  G R R  B B W
O O O  G G Y  G R R  B B W
      B Y Y
      B Y Y
      R Y Y

Paso 5 : Costo G = 1 Costo H = 6
      W W W
      W W W
      G 

Paso 34 : Costo G = 2 Costo H = 6
      W W W
      W W W
      G G G
B B B  O G G  Y Y Y  R R R
O O O  O G G  R R R  B B B
G G O  R R R  B B B  W W W
      O Y Y
      O Y Y
      O Y Y

Paso 35 : Costo G = 2 Costo H = 6
      R W W
      B W W
      O W W
G O B  W O O  G G G  Y R R
G O B  W G G  R R R  Y B B
G O B  W R R  B B B  Y O O
      Y Y O
      Y Y G
      Y Y R

Paso 36 : Costo G = 2 Costo H = 6
      B B B
      W W W
      W W W
Y Y Y  O O O  G G G  R B O
O O O  G G G  R R R  R B O
G G G  R R R  W W W  R B O
      Y Y Y
      Y Y Y
      B B B

Paso 37 : Costo G = 2 Costo H = 6
      W W W
      W W W
      G G G
B B B  O G G  Y Y Y  R R R
O O O  O G G  R R R  B B B
G G O  R R R  B B B  W W W
      Y Y O
      Y Y O
      Y Y O

Paso 38 : Costo G = 2 Costo H = 6
      B B B
      W W W
      W W W
Y Y Y  O O O  G G G  O B R
O O O  G G G  R R R  O B R
G G G  R R R  W W W  O B R
      Y Y Y
      Y Y Y
      B B B

Paso 39 : Costo G = 2 Costo H = 6
      W W W
      W W W
  

Paso 74 : Costo G = 2 Costo H = 6
      R W O
      B W G
      B W G
B O O  W O Y  R R G  Y R W
B O O  W G Y  R R G  Y B W
B O O  W G Y  R R G  Y B W
      B Y O
      B Y G
      R Y G

Paso 75 : Costo G = 2 Costo H = 6
      W W O
      W W G
      W W G
B B B  O O Y  G R R  R R W
O O O  G G Y  G R R  B B W
Y G G  G R R  W B B  O O O
      Y Y Y
      Y Y Y
      B B R

Paso 76 : Costo G = 2 Costo H = 6
      R W O
      B W G
      B W G
O O B  W O Y  R R G  Y R W
O O B  W G Y  R R G  Y B W
O O B  W G Y  R R G  Y B W
      B Y O
      B Y G
      R Y G

Paso 77 : Costo G = 2 Costo H = 6
      B B B
      W W G
      W W G
R Y Y  O O Y  R R G  W W W
O O O  G G Y  R R G  R B B
O O O  G G Y  W W O  R B B
      B Y Y
      B Y Y
      G R R

Paso 78 : Costo G = 2 Costo H = 4
      W W W
      W W W
      W W W
R R R  B B B  O O O  G G G
O O O  G G G  R R R  B B B
G G G  R R R  B B B  O O O
      Y Y Y
      Y Y Y
      Y Y Y

Paso 79 : Costo G = 2 Costo H = 6
      W W O
      W W G
  

Paso 111 : Costo G = 2 Costo H = 6
      B B B
      W W W
      G G G
Y Y Y  O G G  Y Y Y  B B R
O O O  O G G  R R R  B B R
W W W  O G G  W W W  B B R
      O O O
      Y Y Y
      R R R

Paso 112 : Costo G = 2 Costo H = 6
      W W G
      W W G
      W W G
R R R  B B B  O G G  Y Y Y
O O O  G G O  R R R  B B B
W W W  G G O  R R R  B B B
      O O O
      Y Y Y
      Y Y Y

Paso 113 : Costo G = 2 Costo H = 6
      W W O
      W W O
      G G O
B B B  G G O  Y R R  R R G
O O O  G G Y  Y R R  B B W
W W W  G G Y  Y R R  B B W
      B O O
      B Y Y
      R Y Y

Paso 114 : Costo G = 2 Costo H = 6
      R W O
      B W G
      B W G
B O O  W O Y  G R R  Y R W
B O O  W G Y  G R R  Y B W
B O O  W G Y  G R R  Y B W
      B Y O
      B Y G
      R Y G

Paso 115 : Costo G = 2 Costo H = 6
      G W W
      G W W
      G W W
R R R  B B B  O G G  Y Y Y
O O O  G G O  R R R  B B B
W W W  G G O  R R R  B B B
      O O O
      Y Y Y
      Y Y Y

Paso 116 : Costo G = 2 Costo H = 6
      B B B
      W 

      R Y Y

Paso 148 : Costo G = 2 Costo H = 6
      W W B
      W W B
      W W B
B B R  Y Y Y  O O O  G G G
O O O  G G G  R R R  R B B
O O O  G G G  W W W  R B B
      Y Y Y
      Y Y Y
      R R R

Paso 149 : Costo G = 2 Costo H = 6
      R W O
      B W G
      B W G
O O B  W O Y  R R G  Y R W
O O B  W G Y  R R G  Y B W
O O B  W G Y  R R G  Y B W
      B Y O
      B Y G
      R Y G

Paso 150 : Costo G = 2 Costo H = 6
      R W W
      B W W
      O W W
B O G  W O O  G G G  Y R R
B O G  W G G  R R R  Y B B
B O G  W R R  B B B  Y O O
      Y Y O
      Y Y G
      Y Y R

Paso 151 : Costo G = 2 Costo H = 6
      R W W
      B W W
      B G G
B O W  G G G  Y Y Y  Y R R
B O W  W G G  R R R  Y B B
B O W  W G G  R R R  O B B
      O O O
      Y Y O
      Y Y O

Paso 152 : Costo G = 2 Costo H = 6
      Y W W
      Y W W
      Y W W
B B B  B O O  G G G  G R R
O O O  B G G  R R R  G B B
O O O  R G G  R R R  O B B
      Y Y W
      Y Y W
      Y Y W

Paso 153 : Costo G = 2 Costo H = 6
      Y

...

Paso 632 : Costo G = 3 Costo H = 6
      R W W
      B W W
      O W W
B O G  W O O  G G G  Y R R
B O G  W G G  R R R  Y B B
R R W  B B B  O O Y  B O G
      Y Y Y
      Y Y Y
      R G O

Paso 633 : Costo G = 3 Costo H = 6
      W W W
      W W W
      R B O
R R Y  B O G  O O W  G G G
B O G  W G G  R R R  Y B B
B O G  W R R  B B B  Y O O
      Y Y O
      Y Y G
      Y Y R

Paso 634 : Costo G = 3 Costo H = 6
      W W W
      W W W
      Y Y Y
B B B  R O O  Y Y O  R R R
O O O  R G G  R R R  B B B
G G G  R G G  B B B  W W W
      O G G
      Y Y O
      Y Y O

Paso 635 : Costo G = 3 Costo H = 6
      R W Y
      B W Y
      B W Y
B O O  W O B  R R R  Y R G
B O O  W G B  R R R  Y B G
B O O  W G R  G G G  Y B O
      W Y O
      W Y G
      W Y G

Paso 636 : Costo G = 3 Costo H = 6
      O B R
      W W W
      W W W
R R Y  B O G  O O W  G G G
B O G  W G G  R R R  Y B B
B O G  W R R  B B B  Y O O
      Y Y O
      Y Y G
      Y Y R

Paso 637 : Costo G = 3 Costo H = 6
      R W Y
      B 

G G G  W G G  R R R  Y B B
      W W B
      Y Y G
      Y Y G

Paso 673 : Costo G = 3 Costo H = 6
      B B O
      W W G
      W W R
Y Y Y  O O Y  W R G  R B W
O O O  G G Y  W R G  R B W
G G G  R R B  W R G  R B B
      O Y Y
      O Y Y
      O B B

Paso 674 : Costo G = 3 Costo H = 6
      R W W
      B W O
      G G O
B O O  W W O  O R R  Y R G
B O O  G G Y  Y R R  Y B W
B W W  G G Y  Y R R  Y B W
      B O B
      B Y G
      R Y G

Paso 675 : Costo G = 3 Costo H = 6
      W W O
      W W G
      W W B
B B B  O O Y  G R O  R R W
O O O  G G Y  G R O  B B W
R R R  B B Y  G R O  G G W
      G Y Y
      B Y Y
      R Y Y

Paso 676 : Costo G = 3 Costo H = 6
      B B R
      W W W
      G G O
W R Y  B O O  Y O W  R R G
B O O  W G Y  R R G  Y B W
B O O  W G Y  R R G  Y B W
      B Y O
      B Y G
      R Y G

Paso 677 : Costo G = 3 Costo H = 6
      W W B
      W W B
      W W B
B B R  Y Y Y  O O O  G G G
O O O  G G G  R R R  R B B
G G G  W W W  B B R  O O O
      R Y Y
      R Y Y
    

Paso 710 : Costo G = 3 Costo H = 6
      W W Y
      W W Y
      W W Y
B B B  O O B  R R R  R R G
O O O  G G B  R R R  B B G
R G G  G G G  O B B  O O O
      W W W
      Y Y Y
      Y Y Y

Paso 711 : Costo G = 3 Costo H = 6
      R B B
      B W W
      B W W
R R R  Y Y Y  O O O  R R G
O O O  G G G  R R R  B B G
O O O  G G G  B W W  B B G
      Y Y Y
      Y Y Y
      W W W

Paso 712 : Costo G = 3 Costo H = 6
      B W W
      B W W
      O O O
B B R  Y G G  Y Y Y  G G G
O O O  Y G G  R R R  R B B
B W W  Y G G  W W W  R B B
      O O O
      Y Y Y
      R R R

Paso 713 : Costo G = 3 Costo H = 6
      B B B
      W W W
      G G G
Y Y Y  O G R  Y Y Y  R B O
O O O  O G R  R R R  R B O
W W W  O G R  W W W  R B O
      G G G
      Y Y Y
      B B B

Paso 714 : Costo G = 3 Costo H = 6
      Y Y Y
      W W W
      W W W
G R R  B B B  B O O  R R R
O O O  G G B  R R R  B B G
O O O  G G R  G G G  B B O
      W Y Y
      W Y Y
      W Y Y

Paso 715 : Costo G = 3 Costo H = 6
      B W W
      B 

Paso 743 : Costo G = 3 Costo H = 6
      W W O
      W W G
      Y Y B
B B B  O O Y  O O O  R R W
O O O  G G Y  G R R  B B W
R R G  G G Y  G R R  B B W
      G W W
      B Y Y
      R Y Y

Paso 744 : Costo G = 3 Costo H = 6
      B W W
      B W W
      B W W
B B R  Y Y Y  O O O  G G G
O O O  G G G  R R R  R B B
G G G  W W W  B B R  O O O
      R Y Y
      R Y Y
      R Y Y

Paso 745 : Costo G = 3 Costo H = 6
      W W B
      W W B
      W W B
B B R  Y Y Y  O O O  G G G
O O O  G G G  R R R  R B B
G G G  W W W  B B R  O O O
      R Y Y
      R Y Y
      R Y Y

Paso 746 : Costo G = 3 Costo H = 6
      B W W
      B W W
      B W W
O B R  Y Y Y  O O O  G G G
O O O  G G G  R R R  R B O
G G G  R R R  W W W  R B O
      Y Y Y
      Y Y Y
      B B B

Paso 747 : Costo G = 3 Costo H = 6
      W W O
      W W G
      Y Y B
B B B  Y G G  O O O  R R W
O O O  Y G G  G R R  B B W
R R G  Y O O  G R R  B B W
      G W W
      B Y Y
      R Y Y

Paso 748 : Costo G = 3 Costo H = 6
      W W R
      W 

W R R  B B B  O G G  B Y Y
O O O  G G O  G R R  B B W
W W G  Y Y Y  G R R  B B W
      O O O
      B Y Y
      R Y Y

Paso 763 : Costo G = 3 Costo H = 4
      W W W
      W W W
      W W W
G G G  R R R  B B B  O O O
O O O  G G G  R R R  B B B
G G G  R R R  B B B  O O O
      Y Y Y
      Y Y Y
      Y Y Y

Paso 764 : Costo G = 3 Costo H = 0
      W W W
      W W W
      W W W
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
O O O  G G G  R R R  B B B
      Y Y Y
      Y Y Y
      Y Y Y

Solución encontrada: [<__main__.CuboRubik object at 0x000002499BA4D850>, <__main__.CuboRubik object at 0x000002499BA68A00>, <__main__.CuboRubik object at 0x000002499BA61400>, <__main__.CuboRubik object at 0x000002499CF4ADF0>]
